### Application: Develop a process to compute potential insurance payouts informed by conflict return period thresholds.

#### Contextual information

<span style="color:red">Guidance provided by Håvard</span>

1. Start out with a dataset with rows for each grid cell year 
2. Identify the cells for a country for each year that qualify to each of the thresholds and set the payout rate as a value in a column for each cell year 
3. Multiply the payout with the population in the cell in a new column to get the population-weighted payout 
4. Sum the population-weighted payout for each country year 
5. Divide the summed population-weighted payout for each country year by the country’s total population 

<span style="color:red">Sentence structure example provided by Jerry</span>

To make this a bit more clear, let's assume you have two grids that have 100 percent payout rate. These grids each have 1 percent of the countries population. The national number is = 100 x.01 + 100 x .01 or 2 percent. That 2 percent value represents the payout rate for the nation. The protection level of $1M for the national would pay 2 percent of $1M.  

<span style="color:red">Key difference to explore</span>

what effect does the order of operations have, to adjust calculating proportional population as the third task rather than fifth, on the resulting payout metric?


### Locate necessary files:

The function used to generate the complete payout table requires 5 parameters. To keep this focus constrained to the processes generating the final payout value, preceding steps to compute the necessary tables are ommitted. If you have an interest in exploring this information, the Benz_Graphics branch is updated and contains an .ipynb referenced to generate all tables and infographics. 

**base files relevant for this review:**
- x: <span style="color:lightblue">/.../VIEWS_FAO_index/notebooks/methods/Proof_For_Summary_Table/Example_dataframe.csv</span>
- y: <span style="color:lightblue">/.../VIEWS_FAO_index/notebooks/methods/Proof_For_Summary_Table/y__annual_summary_intensity.csv</span>
- z: <span style="color:lightblue">/.../VIEWS_FAO_index/notebooks/methods/Proof_For_Summary_Table/z__raw_insurance_table.csv</span>
- info_df: <span style="color:lightblue">/.../VIEWS_FAO_index/notebooks/methods/Proof_For_Summary_Table/Example_return_period_ranges.csv</span>

#### Address the requirement for multiple input tables:

- `x` is the main DataFrame. It contains the most granular data, with fields such as [pg_id, year, fatalities_sum, pop_gpw_sum, percapita_100k]. This represents the most disaggregated information.

- `y` contains summary statistics, including fields like 'max' and 'average'. This table was initially requested by Jerry on XX to complement specific analysis needs.

- `z` is the original, unrevised "insurance payout table." It communicates the floor thresholds associated with each return period and the respective payout rates. 

- `info_df` is used primarily for formatting graphics and aids in deriving a range that facilitates feature engineering.

These seemingly arbitrary names are intentionally chosen. Using easily distinguishable variable names facilitates the differentiation between tables that contain closely related information. For instance, the term "annual table" could apply to both **`y`** and the resulting payout table, which may lead to confusion. Similarly, a variable named "insurance" could be misinterpreted as referring to either **`info_df`** or **`z`**. Each table contains unique fields that need to remain distinguishable, yet all are collectively integrated in the final payout table through the `append_return_periods_to_annual_table` function.

Table y offers the least intuitive contribution to the final payout table; We aim to retain the ability to sort the comprehensive table by magnitude or intensity, ensuring flexibility in analysis.

#### Load the csv files

In [3]:
import os
import pandas as pd

#SET PATH TO FILES
#--------------------------------------------------------------------------------------------
main_dir = os.getcwd()
#--------------------------------------------------------------------------------------------
#Load files:
example_dataframe_path = os.path.join(main_dir, 'Example_dataframe.csv')
example_return_period_ranges = os.path.join(main_dir, 'Example_return_period_ranges.csv')
#--------------------------------------------------------------------------------------------
original_insurance_table = os.path.join(main_dir, 'z__raw_insurance_table.csv')
example_return_period_ranges = os.path.join(main_dir, 'Example_return_period_ranges.csv')
#--------------------------------------------------------------------------------------------
annual_intensity_table = os.path.join(main_dir, 'y__annual_summary_intensity.csv')
#--------------------------------------------------------------------------------------------
# access the files
x = pd.read_csv(example_dataframe_path, index_col=None)
y = pd.read_csv(annual_intensity_table, index_col=None)
#--------------------------------------------------------------------------------------------
z = pd.read_csv(original_insurance_table, index_col=None)
filtered_info = pd.read_csv(example_return_period_ranges, index_col=None)
#--------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------
print('This provides a review of x:')
print()
display(x.head(5))
print()
print('This provides a review of y')
display(y)
print()
print('This provides a review of z:')
display(z)
print()
print('This provides a review of filtered_info:')
display(filtered_info)

This provides a review of x:



,Unnamed: 0,pg_id,year,fatalities_sum,pop_gpw_sum,percapita_100k
0,0,135077,1993,0.0,15539.840820,0.0
1,1,135077,1994,0.0,16002.742188,0.0
2,2,135077,1995,0.0,16465.642578,0.0
3,3,135077,1996,0.0,16904.835938,0.0
4,4,135077,1997,0.0,17344.029297,0.0



This provides a review of y


,Unnamed: 0,year,first_value,second_value,third_value,average_value
0,0,1993,1001.0,36.0,35.0,2.112597
1,1,1994,50.0,6.0,1.0,0.105850
2,2,1995,1000.0,4.0,2.0,1.815305
3,3,1996,53.0,13.0,3.0,0.126491
4,4,1997,3.0,3.0,3.0,0.020539
5,5,1998,350.0,350.0,105.0,1.646984
6,6,1999,2012.0,311.0,311.0,4.476562
7,7,2000,121.0,104.0,65.0,0.662595
8,8,2001,43.0,40.0,36.0,0.429151
9,9,2002,190.0,170.0,75.0,1.100403



This provides a review of z:


,Unnamed: 0,Percentile,fatalities_sum,Occurrence,Return Period,Payout Rate
0,0,90,0.0,0,10.0,30%
1,1,95,0.0,10017,20.0,55%
2,2,96,1.0,214,25.0,undefined
3,3,98,8.0,108,50.0,75%
4,4,99,31.0,105,100.0,100%
5,5,max,2012.0,1,--,--



This provides a review of filtered_info:


,Unnamed: 0,Return Period,Range,Label
0,0,0,0 - 0.0,Below 1 in 10 year
1,1,10,0.0 - 1.0,1 in 10 year
2,2,20,1.0 - 8.0,1 in 20 year
3,3,50,8.0 - 31.0,1 in 50 year
4,4,100,31.0 - 100000,1 in 100 year


### Load the function: `append_return_periods_to_annual_table`

In addition to the previously defined DataFrames x, y, z, and filtered_info, two additional parameters must be specified when using this function.

- **Attribute Designation:** The user is required to identify the attribute that corresponds to the construction of the x, y, z, and filtered_info tables. In this application, the relevant field used is <span style="color:red">'fatalities_sum'</span> .

- **Population Field Reference:** The user must also provide the name of the population field. This is important as it ensures that the code remains adaptable for scenarios where different population data might be utilized in the future, requiring a unique field name distinct from the one currently being evaluated. The field for this use-case is <span style="color:red"> pop_gpw_sum </span> 

- **Return Period Statistic** Similarly, designate the return period type. The parameter expects one of two strings:
    - Country year -- this is the 'little p' method 
    - Event year -- this is the 'big p' method
    - This analysis employs the 'little p' method so <span style="color:red">Country year</span> will be inserted as the parameter

Information on these methods is present in the glossary markdown file located in VIEWS_FAO_INDEX/docs

In [4]:
from generate_payout_table import append_return_periods_to_annual_table

#### Run the function:

Proceeding steps will then break down the two processes considered

In [5]:
table = append_return_periods_to_annual_table(x, y, z, filtered_info, 'fatalities_sum', 'pop_gpw_sum', 'Country year')

display(table)

you are working with the country year -- little p --  return period type...
{'10.0': '30%', '20.0': '55%', '25.0': 'undefined', '50.0': '75%', '100.0': '100%', '--': '--'}
['Unnamed: 0', 'count_10', 'count_100', 'count_20', 'count_50', 'pop prop_10', 'pop prop_100', 'pop prop_20', 'pop prop_50', 'pop sum_10', 'pop sum_100', 'pop sum_20', 'pop sum_50', 'pay weight 10', 'pay weight 20', 'pay weight 50', 'pay weight 100']
['count_100', 'pop prop_100', 'pop sum_100', 'pay weight 100', 'count_50', 'pop prop_50', 'pop sum_50', 'pay weight 50', 'count_20', 'pop prop_20', 'pop sum_20', 'pay weight 20', 'count_10', 'pop prop_10', 'pop sum_10', 'pay weight 10', 'Unnamed: 0']


,year,first_value,average_value,count_100,pop prop_100,pop sum_100,pay weight 100,count_50,pop prop_50,pop sum_50,...,pop sum_20,pay weight 20,count_10,pop prop_10,pop sum_10,pay weight 10,Unnamed: 0,weight_rp,payout rate (%),Total Payout
0,1993,1001.0,2.112597,3,0.001282,6.853750e+04,0.128248,2,0.004828,2.580117e+05,...,2.635229e+04,0.027121,395,0.993396,5.308843e+07,29.801894,0,4370,30.319359,303193.587952
1,1994,50.0,0.105850,1,0.000345,1.859967e+04,0.034540,0,0.000000,0.000000e+00,...,4.420798e+05,0.451522,369,0.991445,5.338918e+07,29.743353,1,3830,30.229415,302294.151281
2,1995,1000.0,1.815305,1,0.004749,2.631589e+05,0.474864,0,0.000000,0.000000e+00,...,1.779443e+06,1.766031,369,0.963142,5.337508e+07,28.894251,2,3830,31.135147,311351.465864
3,1996,53.0,0.126491,1,0.000514,2.926424e+04,0.051412,1,0.000295,1.680782e+04,...,1.861178e+06,1.798365,367,0.966493,5.501378e+07,28.994791,3,3880,30.866715,308667.148413
4,1997,3.0,0.020539,0,0.000000,0.000000e+00,0.000000,0,0.000000,0.000000e+00,...,2.372922e+06,2.233840,368,0.959385,5.605145e+07,28.781542,4,3760,31.015382,310153.820188
5,1998,350.0,1.646984,6,0.025061,1.501871e+06,2.506137,1,0.000277,1.661267e+04,...,5.017793e+05,0.460519,362,0.966288,5.790746e+07,28.988650,5,4330,31.976097,319760.971850
6,1999,2012.0,4.476562,5,0.010579,6.498972e+05,1.057929,2,0.000759,4.664040e+04,...,7.126682e+05,0.638061,362,0.977060,6.002186e+07,29.311811,6,4280,31.064743,310647.434694
7,2000,121.0,0.662595,5,0.006855,4.314453e+05,0.685548,4,0.003624,2.280991e+05,...,3.361044e+05,0.293730,359,0.984180,6.193876e+07,29.525387,7,4370,30.776495,307764.947680
8,2001,43.0,0.429151,5,0.009134,5.895391e+05,0.913363,3,0.000575,3.712158e+04,...,2.273640e+05,0.193738,359,0.986769,6.369198e+07,29.603062,8,4340,30.753297,307532.969207
9,2002,190.0,1.100403,6,0.002699,1.785833e+05,0.269936,5,0.023833,1.576763e+06,...,2.929727e+06,2.435623,352,0.929183,6.147252e+07,27.875494,9,4550,32.368562,323685.616409
